# Testing

This notebook is to test functionality of recommendation functions.

# Libraries

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2 #would be where you need to specify the files
#%aimport comic_recs

# Pyspark imports
import pyspark
from pyspark.sql import SparkSession

In [3]:
# spark config
spark = SparkSession \
    .builder \
    .appName("movie recommendation") \
    .config("spark.driver.maxResultSize", "1g") \
    .config("spark.driver.memory", "1g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.master", "local[*]") \
    .getOrCreate()

## Data Prep

These data/modeling related tasks need to be prepared beforehand.

### 1. Retrieve comics list as PySpark dataframe

In [ ]:
### 1. Retrieve comics list as PySpark dataframe

In [4]:
comics_df = spark.read.json('support_data/comics.json')
comics_df.persist()

DataFrame[comic_id: bigint, comic_title: string]

In [6]:
comics_df.show(5)

+--------+--------------------+
|comic_id|         comic_title|
+--------+--------------------+
|       1|0Secret Wars (Mar...|
|       2|100 Bullets Broth...|
|       3|100 Penny Press L...|
|       4|100 Penny Press S...|
|       5|100 Penny Press T...|
+--------+--------------------+
only showing top 5 rows



### 2. Retrieve training data
All comic titles existing users have bought or subscribed.

In [7]:
training_data = spark.read.json('raw_data/als_input_filtered.json')
training_data.persist()

In [9]:
training_data.show(5)

+----------+------+--------+
|account_id|bought|comic_id|
+----------+------+--------+
|      2247|     1|     995|
|       487|     1|    1102|
|        29|     1|    2680|
|      1260|     1|    4870|
|       172|     1|    6023|
+----------+------+--------+
only showing top 5 rows



### 3. Set up model parameters
Parameters we have previously found through grid searching / cross-validation.

## Recommendations for New Users

In [10]:
# Create dictionary of candidate parameters
model_params = {'maxIter': 20
                 ,'rank': 10
                 ,'regParam': 0.1
                 ,'alpha': 40
                 ,'seed': 41916
               }

### Create list of existing user preferences

E.g. "I currently like these comic books." Doesn't have to be exact match, but the closer to an actual title, the better.

In [ ]:
reading_list = ['Transformers', 'GI Joe', 'Y The Last Man', 'Saga', 'Avengers'
           ,'Paper Girls', 'Star Wars']

In [ ]:
recommendations = make_comic_recommendations(reading_list
                               ,20
                               ,comics_df
                               ,reading_list
                               ,current_params)